In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_seasonal.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_seasonal.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_seasonal.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_prcptot_seasonal.nc']

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_prcptot_seasonal.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds45 = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds45.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 152)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1949-12-01 1950-12-01 ... 2100-12-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 3, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

In [44]:
i = 0
df45list = []
for ds in listds45:
    print ("Dataset number: ", i)
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df45=0
    df453 = df452.loc[df452["realization"] == 0].dropna()
    df452 = 0
    print ("Converted to DF")
    dfp452 =  df453.reset_index()
    df453 =0
    df45C = dfp452.copy()
    dfp452=0
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45C=0
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    Region1i45["season"] = season
    df45list.append(Region1i45)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list


In [21]:
df45list=0

In [45]:
df45 = pd.concat(df45list)

In [49]:
df45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df45"+var+"_forTS_seasonal.feather")

AttributeError: 'int' object has no attribute 'to_feather'

In [ ]:
df45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df45"+var+"_forTS_seasonal.feather")

# FOR RCP85 

In [10]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 152)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1949-12-01 1950-12-01 ... 2100-12-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 3, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

In [43]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df85=0
    df853 = df852.loc[df852["realization"] == 0].dropna()
    df852=0
    print ("Converted to DF")
    dfp852 =  df853.reset_index()
    df853=0
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    df85C=0
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    Region1i85["season"] = season
    df85list.append(Region1i85)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list


In [20]:
df85list =0

In [46]:
df85 = pd.concat(df85list)

In [ ]:
df85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df85"+var+"_forTS_seasonal.feather")

In [ ]:
df85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df85"+var+"_forTS_seasonal.feather")

# OBSERVED DATA

In [13]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [14]:
dsOBs

<xarray.Dataset>
Dimensions:  (lat: 510, lon: 1068, time: 257)
Coordinates:
  * time     (time) datetime64[ns] 1949-12-01 1950-03-01 ... 2013-12-01
  * lat      (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
  * lon      (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
Data variables:
    prcptot  (time, lat, lon) float32 dask.array<chunksize=(257, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-22T13:14:41: Convert from original format to Net...
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [15]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [22]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})
dfmeanObs=0

In [39]:
season_dict = {12:"winter", 3:"spring", 6:"summer", 9:"fall" }

dfObs["season"]= dfObs.time.dt.month.map(lambda m: season_dict[m])
dfObs

,lat,lon,time,prcptot_Obs,season
0,83.125000,-74.541672,1950-03-01,16.820000,spring
1,83.125000,-74.541672,1950-06-01,20.840002,summer
2,83.125000,-74.541672,1950-09-01,50.200008,fall
3,83.125000,-74.541672,1950-12-01,20.729998,winter
4,83.125000,-74.541672,1951-03-01,16.969997,spring
...,...,...,...,...,...
65602060,41.708332,-82.208336,2012-09-01,166.379974,fall
65602061,41.708332,-82.208336,2012-12-01,208.669983,winter
65602062,41.708332,-82.208336,2013-03-01,190.270004,spring
65602063,41.708332,-82.208336,2013-06-01,350.260071,summer


In [ ]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [ ]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [47]:
df2m =  pd.merge(df45, df85, on=["lat","lon", "time", "season"], suffixes=('_45', '_85'))
df45 = 0
df85 = 0

In [48]:
df2m

,index_45,lat,lon,time,prcptot_p10_45,prcptot_p50_45,prcptot_p90_45,season,index_85,prcptot_p10_85,prcptot_p50_85,prcptot_p90_85
0,1,66.623306,-89.045212,1950-12-01,25.03,34.60,47.51,winter,1,24.52,34.35,47.57
1,2,66.623306,-89.045212,1951-12-01,22.80,30.75,46.68,winter,2,22.42,30.76,46.76
2,3,66.623306,-89.045212,1952-12-01,16.67,29.58,50.60,winter,3,16.10,29.69,50.23
3,4,66.623306,-89.045212,1953-12-01,19.05,25.30,34.58,winter,4,18.61,25.36,34.17
4,5,66.623306,-89.045212,1954-12-01,24.16,33.77,44.65,winter,5,24.09,33.69,44.41
...,...,...,...,...,...,...,...,...,...,...,...,...
45952213,220720470,40.041039,-74.045807,2096-09-01,200.24,284.72,424.18,fall,220720470,206.37,312.75,359.91
45952214,220720471,40.041039,-74.045807,2097-09-01,214.80,299.52,394.04,fall,220720471,179.89,259.75,451.88
45952215,220720472,40.041039,-74.045807,2098-09-01,185.18,218.00,376.83,fall,220720472,138.02,273.69,396.32
45952216,220720473,40.041039,-74.045807,2099-09-01,221.48,326.66,357.05,fall,220720473,211.11,303.41,379.98


In [51]:
df2m.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df2m"+var+"_forTS_seasonal.feather")

In [ ]:
df2m = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df2m"+var+"_forTS_seasonal.feather")

# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [24]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [25]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [26]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [29]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [30]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [31]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [27]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [70]:
dfmean45UA2

,time,NM_REG_FOR,index,lat,lon,index_right,OBJECTID,SUP_KM2,Shape_Leng,Shape_Area,index_45,prcptot_p10_45,prcptot_p50_45,prcptot_p90_45,index_85,prcptot_p10_85,prcptot_p50_85,prcptot_p90_85
0,1950-12-01,ABITIBI-TEMISCAMINGUE,93379.469880,47.944924,-77.942575,6.000000,7.000000,57124.200000,1.961835e+06,5.674050e+10,1.561305e+08,138.485311,164.488363,178.774779,1.561305e+08,137.545984,164.202771,178.665482
1,1950-12-01,BAS-SAINT-LAURENT,92634.675418,48.117288,-68.302802,0.000000,1.000000,27645.700000,1.018112e+06,2.743725e+10,1.548852e+08,163.098592,227.062936,251.098329,1.548852e+08,162.129976,227.439952,251.857518
2,1950-12-01,CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES,98123.779436,47.011463,-70.918931,2.000000,3.000000,35573.900000,1.295862e+06,3.544054e+10,1.640630e+08,196.655091,230.181493,262.415920,1.640630e+08,195.152637,229.857313,263.138988
3,1950-12-01,COTE-NORD,77697.267233,51.118034,-64.726578,7.000000,8.000000,379435.800000,5.360494e+06,3.744732e+11,1.299098e+08,132.874342,204.039925,227.012331,1.299098e+08,132.660610,204.010067,227.760382
4,1950-12-01,ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL,106049.842520,45.419871,-72.542371,4.000000,5.000000,23252.300000,1.123268e+06,2.330905e+10,1.773153e+08,189.582310,232.997900,254.505853,1.773153e+08,190.143517,233.546220,254.152126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2099-12-01,LAURENTIDES,99765.310160,46.672905,-74.970461,12.000000,14.000000,22463.000000,1.305456e+06,2.240745e+10,1.668077e+08,208.940588,263.986952,321.503717,1.668077e+08,220.821283,269.528316,353.937620
1796,2099-12-01,MAURICIE-CENTRE-DU-QUÉBEC,95493.936877,47.531973,-73.531526,3.000000,4.000000,52443.700000,1.903121e+06,5.216317e+10,1.596660e+08,200.222835,251.334053,299.808117,1.596660e+08,206.156401,251.465637,325.855094
1797,2099-12-01,NORD-DU-QUEBEC,77784.447543,51.074087,-75.743994,13.998672,10.999734,866865.308552,8.669244e+06,8.588677e+11,1.300557e+08,140.732730,168.924112,230.508242,1.300557e+08,158.136096,189.751232,223.546537
1798,2099-12-01,OUTAOUAIS,99855.242378,46.651294,-76.466572,5.000000,6.000000,39030.000000,1.677902e+06,3.893718e+10,1.669581e+08,194.005838,241.543537,297.178155,1.669581e+08,208.138537,244.339390,301.630457


In [56]:
seasonlist = ["winter", "spring", "summer", "fall"]

In [72]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    for seas in seasonlist:
        print (seas)
        print ("3/8... Get mean by year for every sub-region for every season")
        dfmeanUA45 = dfdatareg45.loc[dfdatareg45['season'] == seas]
        dfmean45UA = dfmeanUA45.groupby(["time", name, 'season']).mean()
        dfmean45UA2 = dfmean45UA.reset_index()
        dfmeanObs = dfdataregObs.loc[dfdataregObs['season'] == seas]
        dfmeanObs2 = dfmeanObs.groupby(["time", name, 'season']).mean() 
        dfmeanObs3 = dfmeanObs2.reset_index()
        print("Merging 3 cases")
        df3m = pd.merge(dfmean45UA2, dfmeanObs3, how='left', on= ["time", name, "season"] )
        print ("4/8... Split by subregion")
        df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                                   var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs', "season"]]
        if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
            df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
        if (var == "solidprcptot"):
            df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
        df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
        for name2,dfsr in df3ma.groupby(name):
            print ("                   "+name2, name)
            dfsrc = dfsr.copy()
            print ("6/8... 4th Polynomial - rcp45,85")
            years = dfsrc.time.dt.year
            dfsrc["time"] = years#.values
            for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
                if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
                    dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
                if (var == "solidprcptot"):
                    dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
                dfsrc[var+p] = round(dfsrc[var+p], 2) 
            print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
            dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_"+seas+".csv")
        break
    break
break
   
                            
       

------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/8... Merging region with data
winter
3/8... Get mean by year for every sub-region for every season


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)



Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
                   COTE-NORD NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region COTE-NORD
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
                   GASPESIE-ILES-DE-LA-MADELEINE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85

SyntaxError: 'break' outside loop (<ipython-input-72-43f520bb2cc1>, line 44)

In [ ]:
df3m